# OpenAI Agents SDK - 도구 통합

이 튜토리얼은 OpenAI Agents SDK에서 에이전트에 도구(tools)를 통합하는 방법을 다룬다. 기본적인 함수 도구 생성부터 비동기 도구, 에러 핸들링, 구조화된 데이터 반환까지 다양한 도구 패턴을 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv

---

## 1. 기본 도구 통합

이 챕터에서는 `@function_tool` 데코레이터를 사용하여 기본적인 도구를 생성하고 에이전트에 통합하는 방법을 다룬다.

### 학습 내용

- `@function_tool` 데코레이터를 사용한 함수 도구 생성
- 에이전트에 의한 자동 도구 발견 및 실행
- 외부 함수를 호출할 수 있는 에이전트 구축
- 시간, 날씨, 데이터 조회를 위한 간단한 도구 패턴

### 환경 설정

먼저 필요한 모듈을 임포트하고 환경 변수를 검증한다.

In [1]:
import os
from dotenv import load_dotenv
from agents import function_tool, Agent, Runner
from datetime import datetime

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### 기본 도구 생성

`@function_tool` 데코레이터를 사용하면 일반 Python 함수를 에이전트가 호출할 수 있는 도구로 변환할 수 있다. 함수의 docstring은 에이전트가 도구의 용도를 이해하는 데 사용되므로 명확하게 작성해야 한다.

In [2]:
@function_tool
def get_current_time() -> str:
    """현재 시간을 사람이 읽기 쉬운 형식으로 반환한다."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@function_tool
def get_weather(city: str) -> str:
    """도시의 현재 날씨 정보를 반환한다."""
    # 실제 구현에서는 날씨 API를 호출할 것이다
    # 데모 목적으로 모의 응답을 반환한다
    weather_data = {
        "서울": "맑음, 기온 18°C",
        "도쿄": "구름 조금, 기온 22°C",
        "뉴욕": "흐림, 기온 15°C",
        "런던": "비, 기온 12°C"
    }
    return weather_data.get(city, f"{city}의 날씨: 맑음, 기온 20°C")

print("도구가 정의되었다: get_current_time, get_weather")

도구가 정의되었다: get_current_time, get_weather


### 도구가 있는 에이전트 생성

에이전트를 생성할 때 `tools` 매개변수에 도구 목록을 전달한다. 에이전트는 사용자의 질문에 따라 적절한 도구를 자동으로 선택하고 호출한다.

In [3]:
# 도구가 있는 에이전트 생성
assistant = Agent(
    name="WeatherAssistant",
    instructions="""
    당신은 시간과 날씨 정보를 제공할 수 있는 도움이 되는 어시스턴트입니다.
    사용자가 시간이나 날씨에 대해 물어보면 사용 가능한 도구를 사용하세요.
    """,
    tools=[get_current_time, get_weather]
)

print(f"에이전트 '{assistant.name}'가 생성되었다.")
print(f"사용 가능한 도구: {len(assistant.tools)}개")

에이전트 'WeatherAssistant'가 생성되었다.
사용 가능한 도구: 2개


### 도구 자동 실행

에이전트에게 질문하면 필요한 경우 자동으로 도구를 호출하고 결과를 응답에 통합한다.

In [5]:
# 에이전트가 도구를 자동으로 사용
print("=== 도구 자동 실행 테스트 ===")

result = await Runner.run(assistant, "지금 몇 시인가요? 그리고 서울 날씨는 어때요?")

print(f"\n[질문] 지금 몇 시인가요? 그리고 서울 날씨는 어때요?")
print(f"[응답] {result.final_output}")

# 도구 호출 확인
tool_calls = [item for item in result.new_items if item.type == "tool_call_item"]
print(f"\n사용된 도구 수: {len(tool_calls)}")

=== 도구 자동 실행 테스트 ===

[질문] 지금 몇 시인가요? 그리고 서울 날씨는 어때요?
[응답] 지금 시간은 2025년 12월 13일 00시 7분입니다.
서울의 현재 날씨는 맑고, 기온은 20°C입니다.

사용된 도구 수: 2


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


In [7]:
# 다양한 질문 테스트
print("=== 다양한 질문 테스트 ===")

questions = [
    "도쿄 날씨 알려줘",
    "현재 시간이 궁금해요",
    "런던이랑 뉴욕 날씨를 비교해줘"
]

for question in questions:
    result = await Runner.run(assistant, question)
    tool_count = len([item for item in result.new_items if item.type == "tool_call_item"])
    print(f"\n[Q] {question}")
    print(f"[A] {result.final_output}")
    print(f"[도구 호출: {tool_count}회]")

=== 다양한 질문 테스트 ===

[Q] 도쿄 날씨 알려줘
[A] 도쿄의 현재 날씨는 맑고, 기온은 20°C입니다.
[도구 호출: 1회]

[Q] 현재 시간이 궁금해요
[A] 현재 시간은 2025년 12월 13일 00시 07분입니다. 다른 것이 궁금하신가요?
[도구 호출: 1회]

[Q] 런던이랑 뉴욕 날씨를 비교해줘
[A] 현재 런던과 뉴욕 모두 맑은 날씨이며, 기온도 각각 20°C로 동일합니다. 두 도시의 날씨 조건이 같은 상태입니다!
[도구 호출: 2회]


### 도구 설계 모범 사례

=== 도구 설계 모범 사례 ===

1. 명확한 docstring 작성
   - 에이전트가 도구의 용도를 이해하는 데 사용됨
   - 매개변수와 반환값을 명확히 설명

2. 타입 힌트 사용
   - 매개변수와 반환 타입을 명시
   - 에이전트가 올바른 인자를 전달하는 데 도움

3. 단일 책임 원칙
   - 각 도구는 하나의 명확한 작업만 수행
   - 복잡한 작업은 여러 도구로 분리

4. 예측 가능한 반환값
   - 일관된 형식의 응답 반환
   - 에러 상황도 명확하게 처리

---

## 2. 고급 도구 통합

이 챕터에서는 비동기 도구, 에러 핸들링, 구조화된 데이터 반환 등 고급 도구 패턴을 다룬다.

### 학습 내용

- 에러 핸들링과 검증이 포함된 비동기 도구
- 구조화된 데이터(Dict, 복잡한 객체)를 반환하는 도구
- 다단계 도구 워크플로우와 도구 체이닝
- 로깅과 에러 복구가 포함된 프로덕션용 도구 패턴

In [9]:
import os
import asyncio
import logging
from dotenv import load_dotenv
from agents import function_tool, Agent, Runner
from typing import Optional, Dict, Any


# 로깅 설정
logging.basicConfig(level=logging.INFO)

print("환경 설정 완료. 고급 도구 통합 데모를 시작한다.")

환경 설정 완료. 고급 도구 통합 데모를 시작한다.


### 비동기 도구와 에러 핸들링

비동기 도구는 외부 API 호출이나 데이터베이스 쿼리와 같은 I/O 작업에 적합하다. 에러 핸들링을 포함하여 실패 시에도 유용한 응답을 반환한다.

In [10]:
@function_tool
async def fetch_user_data(user_id: str) -> Dict[str, Any]:
    """
    외부 서비스에서 사용자 데이터를 조회한다.
    에러 핸들링이 포함되어 있다.
    """
    try:
        # 데이터베이스/API 호출 시뮬레이션
        await asyncio.sleep(0.1)  # 비동기 작업 시뮬레이션
        
        # 모의 사용자 데이터베이스
        users = {
            "12345": {
                "name": "홍길동", 
                "tier": "프리미엄", 
                "last_login": "2025-01-15",
                "email": "hong@example.com"
            },
            "67890": {
                "name": "김영희", 
                "tier": "기본", 
                "last_login": "2025-01-10",
                "email": "kim@example.com"
            }
        }
        
        if user_id not in users:
            return {
                "success": False,
                "error": "사용자를 찾을 수 없음", 
                "user_id": user_id
            }
        
        return {
            "success": True, 
            "user": users[user_id]
        }
        
    except Exception as e:
        logging.error(f"사용자 {user_id} 조회 오류: {e}")
        return {
            "success": False,
            "error": f"서비스 이용 불가: {str(e)}"
        }

print("fetch_user_data 도구가 정의되었다.")

fetch_user_data 도구가 정의되었다.


### 검증 도구

입력 데이터를 검증하고 상세한 결과를 반환하는 도구를 생성한다.

In [11]:
import re

@function_tool
def validate_email(email: str) -> Dict[str, Any]:
    """
    이메일 형식과 도메인을 검증한다.
    """
    email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    is_valid_format = bool(re.match(email_pattern, email))
    
    # 일반적인 도메인 확인
    common_domains = ['gmail.com', 'yahoo.com', 'outlook.com', 'naver.com', 'kakao.com']
    domain = email.split('@')[1] if '@' in email else ''
    is_common_domain = domain.lower() in common_domains
    
    return {
        "email": email,
        "is_valid_format": is_valid_format,
        "domain": domain,
        "is_common_domain": is_common_domain,
        "recommended_action": "진행" if is_valid_format else "수정 요청"
    }

@function_tool
def check_identifier_type(identifier: str) -> Dict[str, Any]:
    """
    식별자가 이메일인지 사용자 ID인지 확인하고 기본 검증을 제공한다.
    """
    if '@' in identifier:
        return {
            "type": "email",
            "identifier": identifier,
            "message": "이메일 주소로 보입니다. 상세 검증을 위해 validate_email 도구를 사용하세요."
        }
    else:
        return {
            "type": "user_id", 
            "identifier": identifier,
            "message": "사용자 ID로 보입니다. 사용자 정보 조회를 위해 fetch_user_data 도구를 사용하세요."
        }

print("검증 도구가 정의되었다: validate_email, check_identifier_type")

검증 도구가 정의되었다: validate_email, check_identifier_type


### 다중 도구 에이전트

여러 도구를 조합하여 복잡한 워크플로우를 수행할 수 있는 에이전트를 생성한다. 에이전트의 지시문에서 도구 사용 순서와 방법을 안내한다.

In [12]:
# 다중 도구 에이전트 생성
support_agent = Agent(
    name="AdvancedSupport",
    instructions="""
    당신은 사용자 조회 도구에 접근할 수 있는 고급 고객 지원 에이전트입니다.
    
    사용자가 정보를 제공하면:
    1. check_identifier_type을 사용하여 이메일인지 사용자 ID인지 확인하세요
    2. 이메일 주소에는 validate_email을 사용하세요
    3. 사용자 ID에는 fetch_user_data를 사용하세요
    4. 항상 어떤 검증 단계를 수행하는지 설명하세요
    5. 검증이 실패하면 도움이 되는 에러 메시지를 제공하세요
    
    전문적이고 철저하게 응답하세요.
    """,
    tools=[fetch_user_data, validate_email, check_identifier_type]
)

print(f"에이전트 '{support_agent.name}'가 생성되었다.")
print(f"사용 가능한 도구: {len(support_agent.tools)}개")

에이전트 'AdvancedSupport'가 생성되었다.
사용 가능한 도구: 3개


### 고급 도구 사용 데모

다양한 시나리오에서 에이전트가 여러 도구를 어떻게 조합하여 사용하는지 확인한다.

In [13]:
async def demonstrate_advanced_tools():
    """
    고급 도구 통합을 시연하는 함수이다.
    """
    print("=== 고급 도구 통합 데모 ===")
    
    test_cases = [
        "사용자 12345를 조회해주세요",
        "이 이메일을 검증해주세요: hong.gildong@gmail.com",
        "이 이메일이 유효한가요: invalid.email@",
        "사용자 99999 정보를 찾아주세요"
    ]
    
    for query in test_cases:
        print(f"\n[사용자] {query}")
        result = await Runner.run(support_agent, query)
        print(f"[에이전트] {result.final_output}")
        
        # 도구 사용 세부 정보 표시
        tool_calls = [item for item in result.new_items if item.type == "tool_call_item"]
        if tool_calls:
            print(f"[도구 호출: {len(tool_calls)}회]")
        print("-" * 50)

# 데모 실행
await demonstrate_advanced_tools()

=== 고급 도구 통합 데모 ===

[사용자] 사용자 12345를 조회해주세요


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


[에이전트] "12345"는 이메일이 아니라 사용자 ID임을 확인했습니다. 이 식별자는 사용자 ID로 분류되므로, fetch_user_data를 통해 정보를 조회했습니다.

조회 결과:
- 이름: 홍길동
- 등급: 프리미엄
- 마지막 로그인: 2025-01-15
- 등록 이메일: hong@example.com

추가로 필요한 정보나 다른 요청 사항이 있으시면 말씀해 주세요!
[도구 호출: 2회]
--------------------------------------------------

[사용자] 이 이메일을 검증해주세요: hong.gildong@gmail.com


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


[에이전트] 1. 입력해주신 "hong.gildong@gmail.com"이 사용자 식별자인지 확인한 결과, 정상적인 이메일 주소로 판별되었습니다.
2. 이메일 형식을 검증한 결과:
   - 이메일 형식이 올바릅니다.
   - 도메인은 "gmail.com"으로, 잘 알려진 정상 도메인입니다.

따라서 이 이메일 주소는 일반적인 용도로 사용하기에 적합하며, 추가적인 문제가 없습니다. 
더 필요한 사항이 있다면 말씀해 주세요!
[도구 호출: 2회]
--------------------------------------------------

[사용자] 이 이메일이 유효한가요: invalid.email@


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


[에이전트] 검증 결과, 입력하신 이메일 "invalid.email@"는 올바른 이메일 주소 형식이 아닙니다.  
이메일 주소는 사용자명@도메인 구조여야 하며, 예를 들어 user@example.com과 같은 형식이어야 합니다.

문제점:
- 도메인(예: @gmail.com, @naver.com 등)이 올바르게 입력되어 있지 않습니다.

수정 권장:
- 이메일의 도메인 부분을 반드시 포함해 주세요. (예: invalid.email@gmail.com)

이메일 주소를 다시 확인하신 후 올바른 형식으로 입력해 주세요. 추가로 궁금한 점이 있으시면 언제든 문의해 주세요!
[도구 호출: 1회]
--------------------------------------------------

[사용자] 사용자 99999 정보를 찾아주세요


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


[에이전트] 우선 "99999"가 이메일 주소인지 검사했으며, 이 값은 사용자 ID로 확인되었습니다. 

이후 사용자 ID로 사용자 정보를 조회하는 단계를 진행했습니다.  
하지만 사용자 데이터베이스에서 "99999"에 해당하는 사용자를 찾을 수 없습니다.

확인해보실 사항:
- 입력하신 사용자 ID가 정확한지 다시 한번 검토해 주세요.
- 다른 식별자(이메일 주소 또는 정확한 사용자 ID 등)가 있다면 알려주시면 다시 확인해드릴 수 있습니다.

추가로 필요한 정보가 있다면 언제든 요청해 주세요.
[도구 호출: 2회]
--------------------------------------------------


### 도구 체이닝 패턴

에이전트가 여러 도구를 순차적으로 호출하여 복잡한 작업을 수행하는 패턴을 시연한다.

In [14]:
# 추가 도구 정의 - 도구 체이닝을 위한
@function_tool
def get_user_orders(user_id: str) -> Dict[str, Any]:
    """
    사용자의 최근 주문 내역을 조회한다.
    """
    orders = {
        "12345": [
            {"order_id": "ORD001", "item": "노트북", "status": "배송완료"},
            {"order_id": "ORD002", "item": "마우스", "status": "배송중"}
        ],
        "67890": [
            {"order_id": "ORD003", "item": "키보드", "status": "처리중"}
        ]
    }
    
    if user_id not in orders:
        return {"success": False, "error": "주문 내역 없음"}
    
    return {
        "success": True,
        "user_id": user_id,
        "orders": orders[user_id],
        "total_orders": len(orders[user_id])
    }

@function_tool
def calculate_loyalty_points(user_id: str, tier: str) -> Dict[str, Any]:
    """
    사용자의 등급에 따른 로열티 포인트를 계산한다.
    """
    tier_multipliers = {
        "프리미엄": 2.0,
        "기본": 1.0
    }
    
    base_points = 1000
    multiplier = tier_multipliers.get(tier, 1.0)
    total_points = int(base_points * multiplier)
    
    return {
        "user_id": user_id,
        "tier": tier,
        "base_points": base_points,
        "multiplier": multiplier,
        "total_points": total_points
    }

print("추가 도구가 정의되었다: get_user_orders, calculate_loyalty_points")

추가 도구가 정의되었다: get_user_orders, calculate_loyalty_points


In [15]:
# 도구 체이닝을 위한 에이전트 생성
comprehensive_agent = Agent(
    name="ComprehensiveSupport",
    instructions="""
    당신은 종합적인 사용자 지원을 제공하는 에이전트입니다.
    
    사용자 정보 요청 시:
    1. 먼저 fetch_user_data로 사용자 정보를 조회하세요
    2. 사용자가 존재하면 get_user_orders로 주문 내역을 조회하세요
    3. calculate_loyalty_points로 포인트를 계산하세요
    4. 모든 정보를 종합하여 보고서 형태로 제공하세요
    
    각 단계에서 무엇을 하고 있는지 설명하세요.
    """,
    tools=[fetch_user_data, get_user_orders, calculate_loyalty_points]
)

print(f"종합 지원 에이전트가 생성되었다.")

종합 지원 에이전트가 생성되었다.


In [16]:
# 도구 체이닝 데모
async def demonstrate_tool_chaining():
    print("=== 도구 체이닝 데모 ===")
    print("\n[요청] 사용자 12345의 전체 정보를 조회하고 보고서를 작성해주세요.")
    
    result = await Runner.run(
        comprehensive_agent,
        "사용자 12345의 전체 정보를 조회하고 보고서를 작성해주세요. 사용자 정보, 주문 내역, 포인트를 모두 포함해주세요."
    )
    
    print(f"\n[에이전트 응답]")
    print(result.final_output)
    
    # 도구 호출 분석
    tool_calls = [item for item in result.new_items if item.type == "tool_call_item"]
    print(f"\n[도구 체이닝 분석]")
    print(f"총 도구 호출: {len(tool_calls)}회")

await demonstrate_tool_chaining()

=== 도구 체이닝 데모 ===

[요청] 사용자 12345의 전체 정보를 조회하고 보고서를 작성해주세요.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"



[에이전트 응답]
주문 내역과 로열티 포인트 정보를 모두 확인했습니다.  
지금까지의 진행 과정을 요약하면 다음과 같습니다:

1. 사용자 기본 정보 조회 완료 — 사용자 12345(홍길동, 프리미엄 등급)님을 확인했습니다.
2. 주문 내역 조회 — 최근 주문 2건(노트북, 마우스)이 확인되었습니다.
3. 로열티 포인트 계산 — 프리미엄 등급 기준, 총 2,000 포인트가 적립되어 있습니다.

이제 모든 정보를 종합하여 아래와 같이 사용자 전체 정보 보고서를 작성합니다.

---

### 사용자 종합 정보 보고서

**1. 기본 정보**
- 이름: 홍길동
- 이메일: hong@example.com
- 등급: 프리미엄
- 마지막 로그인: 2025-01-15

**2. 주문 내역 (총 2건)**
- 주문번호: ORD001 | 상품: 노트북 | 상태: 배송완료
- 주문번호: ORD002 | 상품: 마우스 | 상태: 배송중

**3. 로열티 포인트**
- 등급: 프리미엄
- 기본 적립 포인트: 1,000
- 등급 배수: 2.0
- 총 적립 포인트: 2,000

궁금한 사항이 있으시면 언제든 말씀해 주세요.

[도구 체이닝 분석]
총 도구 호출: 3회


### 프로덕션용 도구 패턴

=== 프로덕션용 도구 패턴 ===

1. 에러 핸들링
   - 모든 도구에 try-except 블록 포함
   - 실패 시 명확한 에러 메시지 반환
   - 로깅으로 문제 추적

2. 입력 검증
   - 매개변수 유효성 검사
   - 타입 체크 및 범위 확인
   - 잘못된 입력에 대한 안내 메시지

3. 구조화된 응답
   - Dict 형식으로 일관된 응답 구조
   - success/error 필드 포함
   - 다음 단계 안내 포함

4. 비동기 처리
   - I/O 작업에 async 사용
   - 타임아웃 설정
   - 병렬 처리 가능

5. 로깅과 모니터링
   - 도구 호출 로깅
   - 성능 메트릭 수집
   - 에러율 추적

---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK에서 도구를 통합하는 방법을 다루었다. 다음과 같은 내용을 학습하였다:

1. **기본 도구 생성**: `@function_tool` 데코레이터를 사용한 간단한 도구 생성
2. **도구 자동 실행**: 에이전트가 필요에 따라 도구를 자동으로 선택하고 호출
3. **비동기 도구**: async/await를 사용한 I/O 작업 처리
4. **에러 핸들링**: try-except와 구조화된 에러 응답
5. **구조화된 데이터 반환**: Dict 형식의 상세한 응답
6. **도구 체이닝**: 여러 도구를 순차적으로 조합하여 복잡한 워크플로우 수행
7. **프로덕션 패턴**: 로깅, 검증, 모니터링 등의 모범 사례

이러한 패턴을 적용하면 에이전트의 기능을 무한히 확장하고, 외부 시스템과 통합하며, 복잡한 비즈니스 로직을 처리할 수 있다.